## Setup

In [5]:
import re
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.performance import *
from alphalens.plotting import *
from alphalens.tears import *

In [6]:
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('fivethirtyeight')

### Version conflict

At the time of writing, `zipline` required `pandas<=0.22` so you may need to run `pip install -U zipline` to temporarily downgrade `pandas` or set up a separate environment.

In [ ]:
!pip install -U zipline

## Zipline AlphaFactor Test

Run using jupyter notebook extension

In [8]:
%load_ext zipline

The zipline extension is already loaded. To reload it, use:
  %reload_ext zipline


In [9]:
%%zipline --start 2015-1-1 --end 2018-1-1 --output single_factor.pickle

from zipline.api import (
    attach_pipeline,
    date_rules,
    time_rules,
    order_target_percent,
    pipeline_output,
    record,
    schedule_function,
    get_open_orders,
    calendars
)
from zipline.finance import commission, slippage
from zipline.pipeline import Pipeline, CustomFactor
from zipline.pipeline.factors import Returns, AverageDollarVolume
import numpy as np
import pandas as pd

MONTH = 21
YEAR = 12 * MONTH
N_LONGS = N_SHORTS = 25
VOL_SCREEN = 1000


class MeanReversion(CustomFactor):
    """Compute ratio of latest monthly return to 12m average,
       normalized by std dev of monthly returns"""
    inputs = [Returns(window_length=MONTH)]
    window_length = YEAR

    def compute(self, today, assets, out, monthly_returns):
        df = pd.DataFrame(monthly_returns)
        out[:] = df.iloc[-1].sub(df.mean()).div(df.std())


def compute_factors():
    """Create factor pipeline incl. mean reversion,
        filtered by 30d Dollar Volume; capture factor ranks"""
    mean_reversion = MeanReversion()
    dollar_volume = AverageDollarVolume(window_length=30)
    return Pipeline(columns={'longs': mean_reversion.bottom(N_LONGS),
                             'shorts': mean_reversion.top(N_SHORTS),
                             'ranking': mean_reversion.rank(ascending=False)},
                    screen=dollar_volume.top(VOL_SCREEN))


def exec_trades(data, assets, target_percent):
    """Place orders for assets using target portfolio percentage"""
    for asset in assets:
        if data.can_trade(asset) and not get_open_orders(asset):
            order_target_percent(asset, target_percent)


def rebalance(context, data):
    """Compute long, short and obsolete holdings; place trade orders"""
    factor_data = context.factor_data
    record(factor_data=factor_data.ranking)

    assets = factor_data.index
    record(prices=data.current(assets, 'price'))

    longs = assets[factor_data.longs]
    shorts = assets[factor_data.shorts]
    divest = context.portfolio.positions.keys() - longs.union(shorts)

    exec_trades(data, assets=divest, target_percent=0)
    exec_trades(data, assets=longs, target_percent=1 / N_LONGS)
    exec_trades(data, assets=shorts, target_percent=-1 / N_SHORTS)


def initialize(context):
    """Setup: register pipeline, schedule rebalancing,
        and set trading params"""
    attach_pipeline(compute_factors(), 'factor_pipeline')
    schedule_function(rebalance,
                      date_rules.week_start(),
                      time_rules.market_open(),
                      calendar=calendars.US_EQUITIES)
    context.set_commission(commission.PerShare(cost=.01, min_trade_cost=0))
    context.set_slippage(slippage.VolumeShareSlippage())


def before_trading_start(context, data):
    """Run factor pipeline"""
    context.factor_data = pipeline_output('factor_pipeline')

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2015-01-02 21:00:00+00:00,NaN,0.000000,NaN,-0.000535,NaN,NaN,0.000000e+00,1.000000e+07,0.000,0.000,...,0.000,0.000,0,NaN,1.000000e+07,0.000,0.000,1,[],0.0
2015-01-05 21:00:00+00:00,0.004462,-0.000398,0.003057,-0.018585,0.196720,0.022683,-3.471997e+06,6.528003e+06,3468022.250,3468022.250,...,-1584674.470,-1584674.470,4,-11.224972,1.000000e+07,0.000,0.000,2,"[{'amount': 17921, 'dt': 2015-01-05 21:00:00+0...",0.0
2015-01-06 21:00:00+00:00,0.087450,-0.010128,-0.785849,-0.027829,0.139106,0.027711,2.579878e+05,6.785990e+06,3112732.065,3112732.065,...,-1616701.625,-1616701.625,4,-9.531491,6.528003e+06,3468022.250,3468022.250,3,"[{'amount': 9694, 'dt': 2015-01-06 21:00:00+00...",0.0
2015-01-07 21:00:00+00:00,0.080463,-0.008845,-0.401664,-0.015715,0.206973,0.158167,0.000000e+00,6.785990e+06,3125561.843,3125561.843,...,-1629422.430,-1629422.430,4,-7.198541,6.785990e+06,3112732.065,3112732.065,4,[],0.0
2015-01-08 21:00:00+00:00,0.092816,-0.002474,-0.150125,0.001752,0.236042,0.260213,0.000000e+00,6.785990e+06,3189267.130,3189267.130,...,-1644535.790,-1644535.790,4,-1.754848,6.785990e+06,3125561.843,3125561.843,5,[],0.0
2015-01-09 21:00:00+00:00,0.085714,-0.006238,-0.194007,-0.006276,0.218112,0.268363,0.000000e+00,6.785990e+06,3151628.598,3151628.598,...,-1670536.830,-1670536.830,4,-3.834416,6.785990e+06,3189267.130,3189267.130,6,[],0.0
2015-01-12 21:00:00+00:00,0.078270,-0.007585,-0.143845,-0.014060,0.203322,0.259295,0.000000e+00,6.785990e+06,3138163.975,3138163.975,...,-1659398.310,-1659398.310,4,-4.292170,6.785990e+06,3151628.598,3151628.598,7,[],0.0
2015-01-13 21:00:00+00:00,0.074637,-0.005492,-0.037817,-0.016834,0.188302,0.256684,1.675249e+06,8.461239e+06,1483843.830,1483843.830,...,-1600072.490,-1600072.490,4,-2.891468,6.785990e+06,3138163.975,3138163.975,8,"[{'amount': -31758, 'dt': 2015-01-13 21:00:00+...",0.0
2015-01-14 21:00:00+00:00,0.070191,-0.007529,-0.046796,-0.022769,0.177394,0.257918,4.236868e+04,8.503608e+06,1421106.150,1421106.150,...,-1571634.240,-1571634.240,4,-3.685843,8.461239e+06,1483843.830,1483843.830,9,"[{'amount': -2035, 'dt': 2015-01-14 21:00:00+0...",0.0
2015-01-15 21:00:00+00:00,0.066363,-0.007372,0.008934,-0.031722,0.170558,0.242304,0.000000e+00,8.503608e+06,1422668.199,1422668.199,...,-1546157.361,-1546157.361,4,-3.423078,8.503608e+06,1421106.150,1421106.150,10,[],0.0
